In [107]:
from argilla import init
import os
from dotenv import load_dotenv
load_dotenv("D:\\gitFolders\\python_de_learners_data\\.env")

True

In [108]:
init(
    api_key="owner.apikey",
    api_url="https://kamaljp-argillatest.hf.space",
    extra_headers={"Authorization": f"Bearer {os.environ['HUGGING_FACE_KEY']}"}
)

d:\argenv\Lib\site-packages\argilla\client\client.py:178: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(


In [109]:
hfgilla = 'hfgilla'

In [ ]:
from argilla import Workspace
Workspace.create(name=hfgilla)

## Big Thanks to Gretel Team for the dataset.
#### Excellent visualisation.. ;)

https://huggingface.co/datasets/gretelai/synthetic_text_to_sql

@software{gretel-synthetic-text-to-sql-2024,
  author = {Meyer, Yev and Emadi, Marjan and Nathawani, Dhruv and Ramaswamy, Lipika and Boyd, Kendrick and Van Segbroeck, Maarten and Grossman, Matthew and Mlocek, Piotr and Newberry, Drew},
  title = {{Synthetic-Text-To-SQL}: A synthetic dataset for training language models to generate SQL queries from natural language prompts},
  month = {April},
  year = {2024},
  url = {https://huggingface.co/datasets/gretelai/synthetic-text-to-sql}
}

In [110]:
from datasets import load_dataset

t2sql = load_dataset("gretelai/synthetic_text_to_sql") 

In [111]:
t2sql

DatasetDict({
    train: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 5851
    })
})

In [112]:
for dat in t2sql['test']:
    print(dat)
    break

{
    'id': 1,
    'domain': 'sports',
    'domain_description': 'Extensive data on athlete performance, team management, fan engagement, facility 
operations, and event planning in sports.',
    'sql_complexity': 'subqueries',
    'sql_complexity_description': 'subqueries, including correlated and nested subqueries',
    'sql_task_type': 'analytics and reporting',
    'sql_task_type_description': 'generating reports, dashboards, and analytical insights',
    'sql_prompt': "Which team has the highest number of wins in the 'basketball_games' table?",
    'sql_context': "CREATE TABLE basketball_teams (team_id INT, name VARCHAR(50)); CREATE TABLE basketball_games 
(game_id INT, home_team INT, away_team INT, home_team_score INT, away_team_score INT); INSERT INTO basketball_teams
(team_id, name) VALUES (1, 'Boston Celtics'), (2, 'Los Angeles Lakers'), (3, 'Chicago Bulls'); INSERT INTO 
basketball_games (game_id, home_team, away_team, home_team_score, away_team_score) VALUES (1, 1, 2, 85, 80), (2, 2,
3, 95, 90), (3, 3, 1, 75, 85);",
    'sql': 'SELECT name AS team, MAX(home_team_wins + away_team_wins) AS highest_wins FROM (SELECT name, CASE WHEN 
home_team = team_id AND home_team_score > away_team_score THEN 1 ELSE 0 END + CASE WHEN away_team = team_id AND 
away_team_score > home_team_score THEN 1 ELSE 0 END AS home_team_wins, CASE WHEN home_team = team_id AND 
home_team_score < away_team_score THEN 1 ELSE 0 END + CASE WHEN away_team = team_id AND away_team_score < 
home_team_score THEN 1 ELSE 0 END AS away_team_wins FROM basketball_teams JOIN basketball_games ON 
basketball_teams.team_id = basketball_games.home_team OR basketball_teams.team_id = basketball_games.away_team) AS 
subquery GROUP BY name;',
    'sql_explanation': "The SQL query calculates the number of wins for each team in the 'basketball_games' table 
by using a subquery to join the 'basketball_teams' table with the 'basketball_games' table on the 'team_id' column.
The subquery calculates the number of wins at home and away for each team by using the CASE statement. The outer 
query then uses the MAX function to find the team with the highest number of wins. The query then uses the GROUP BY
clause to group the results by the 'name' column. The query then returns the name of each team and the highest 
number of wins."
}

Following fields can be made into meta_data

- domain, domain_description, sql_complexity,  sql_task_type,

Following fields can be used for Vector based search / similarity filtering

- sql_complexity_description, sql_task_type_description, sql_prompt, sql_context

In [ ]:
from argilla import FeedbackDataset, FeedbackRecord
from argilla.client.feedback.integrations.sentencetransformers import SentenceTransformersExtractor

gretel_path = "gretelai/synthetic_text_to_sql"
hf2arg_fbds =  FeedbackDataset.from_huggingface(gretel_path)  # doesn't work
# as the above dataset is not configured based on argilla DS requirement

# local2server_fbds = hf2arg_fbds.push_to_argilla(name='text2sql', workspace=hfgilla)

In [122]:
# Preparing the Feedback Dataset parts
from argilla import TermsMetadataProperty

meta = TermsMetadataProperty(
    name='domain',
    title="meta data about the sql prompt and sql query",
    # values= ["domain", "domain_description", "sql_complexity", "sql_task_type"]
)

In [123]:
# Preparing the Dataset parts

meta_domain = TermsMetadataProperty(
    name='domain',
    title="domain of prompt",
    # values= ["domain"]
)

meta_domain_desc = TermsMetadataProperty(
    name='domain_description',
    title="domain explanation",
    # values= ["domain_description"]
)

meta_sql_complexity = TermsMetadataProperty(
    name='sql_complexity',
    title="Complexity level of SQL query",
    # values= ["sql_complexity"]
)
meta_sql_task = TermsMetadataProperty(
    name='sql_task_type',
    title="type of sql query task",
    # values= ["sql_task_type"]
)

In [48]:
from argilla.client.feedback.integrations.sentencetransformers import SentenceTransformersExtractor

ste = SentenceTransformersExtractor(
    model="TaylorAI/bge-micro-v2",
    show_progress=True
)

In [113]:
from argilla import TextQuestion
quest = TextQuestion(
    name='sqltext',
    title='Checking output of the sql query, sql explanation',
    description="Review the SQL query field and provide feedback",
    required=True,
    use_markdown=True
)

In [132]:
from argilla import (
    FeedbackDataset,
    FeedbackRecord,
    TextField,
)
# sql_complexity_description, sql_task_type_description, sql_prompt, sql_context
t2sql_fbds = FeedbackDataset(
    fields=[
        TextField(name='sql_complexity_description', title='Sql Complexity', required=True, use_markdown=True),
        TextField(name='sql_task_type_description', title='Task Description', required=True, use_markdown=True),
        TextField(name='sql_prompt', title='prompt', required=True, use_markdown=True),
        TextField(name='sql_context', title='context', required=True, use_markdown=True),
        TextField(name='sql', title='SQL Query', required=True, use_markdown=True),
        TextField(name='sql_explanation', title='SQL Explanation', required=True, use_markdown=True),
    ],
    questions=[quest],
    # metadata_properties=[meta],  # If there is no metadata properties, not reflected in UI
    metadata_properties=[meta_domain, meta_domain_desc, meta_sql_complexity, meta_sql_task],
    # vectors_settings=vector_set,
    allow_extra_metadata=True
)

In [133]:
t2sql_fbds.push_to_argilla(name="t2sql_ds", workspace=hfgilla)

[04/06/24 16:13:25] INFO     INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully  ]8;id=389293;file://d:\argenv\Lib\site-packages\argilla\client\feedback\dataset\local\mixins.py\mixins.py]8;;\:]8;id=184092;file://d:\argenv\Lib\site-packages\argilla\client\feedback\dataset\local\mixins.py#271\271]8;;\
                             pushed to Argilla                                                                     

                    INFO     INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset( ]8;id=702859;file://d:\argenv\Lib\site-packages\argilla\client\feedback\dataset\local\mixins.py\mixins.py]8;;\:]8;id=987122;file://d:\argenv\Lib\site-packages\argilla\client\feedback\dataset\local\mixins.py#272\272]8;;\
                                id=94f8c950-37a7-404e-8831-c2d43245f039                                            
                                name=t2sql_ds                                                                      
                                workspace=Workspace(id=eb6c560d-627d-42e1-bdbf-ace020199880,                       
                             name=hfgilla, inserted_at=2024-04-06 04:22:46.479068,                                 
                             updated_at=2024-04-06 04:22:46.479068)                                                
                                url=https://kamaljp-argillatest.hf.space/dataset/94f8c950-37a7-404e-8              
                             831-c2d43245f039/annotation-mode                                                      
                                fields=[RemoteTextField(id=UUID('e3b5577a-9474-4fe7-bae6-8267d024bb73              
                             '), client=None, name='sql_complexity_description', title='Sql                        
                             Complexity', required=True, type='text', use_markdown=True),                          
                             RemoteTextField(id=UUID('b3030561-76ad-4236-8d7e-e7eed4fb294b'),                      
                             client=None, name='sql_task_type_description', title='Task Description',              
                             required=True, type='text', use_markdown=True),                                       
                             RemoteTextField(id=UUID('b7b169a0-4ac4-41ca-9813-c1a4964d0a73'),                      
                             client=None, name='sql_prompt', title='prompt', required=True,                        
                             type='text', use_markdown=True),                                                      
                             RemoteTextField(id=UUID('0c0d94df-3e23-4e5e-80bd-0eed82bba4a9'),                      
                             client=None, name='sql_context', title='context', required=True,                      
                             type='text', use_markdown=True),                                                      
                             RemoteTextField(id=UUID('6d2714e7-5923-4d02-8115-92f6921a65da'),                      
                             client=None, name='sql', title='SQL Query', required=True, type='text',               
                             use_markdown=True),                                                                   
                             RemoteTextField(id=UUID('bb316453-69d8-4919-a2c6-6832ead8be41'),                      
                             client=None, name='sql_explanation', title='SQL Explanation',                         
                             required=True, type='text', use_markdown=True)]                                       
                                questions=[RemoteTextQuestion(id=UUID('f60dadc7-db0d-4f75-b2aa-7f041d              
                             e812d1'), client=None, name='sqltext', title='Checking output of the sql              
                             query, sql explanation', description='Review the SQL query field and                  
                             provide feedback', required=True, type='text', use_markdown=True)]                    
                                guidelines=None                                                                    
                                metadata_properties=[RemoteTermsMetadataProperty(id=UUID('3143944d-8d              
                             1a-487d-925c-ae8f21607d52'), client=<httpx.Cl

RemoteFeedbackDataset(
   id=94f8c950-37a7-404e-8831-c2d43245f039
   name=t2sql_ds
   workspace=Workspace(id=eb6c560d-627d-42e1-bdbf-ace020199880, name=hfgilla, inserted_at=2024-04-06 04:22:46.479068, updated_at=2024-04-06 04:22:46.479068)
   url=https://kamaljp-argillatest.hf.space/dataset/94f8c950-37a7-404e-8831-c2d43245f039/annotation-mode
   fields=[RemoteTextField(id=UUID('e3b5577a-9474-4fe7-bae6-8267d024bb73'), client=None, name='sql_complexity_description', title='Sql Complexity', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('b3030561-76ad-4236-8d7e-e7eed4fb294b'), client=None, name='sql_task_type_description', title='Task Description', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('b7b169a0-4ac4-41ca-9813-c1a4964d0a73'), client=None, name='sql_prompt', title='prompt', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('0c0d94df-3e23-4e5e-80bd-0eed82bba4a9'), client=None, name='sql_context', title='context

In [134]:
# run delete only when required
serside_t2sql = FeedbackDataset.from_argilla(name="t2sql_ds", workspace=hfgilla)

In [131]:
serside_t2sql.delete()

In [118]:
# No metadata involved
t2sql_records = [FeedbackRecord(
    fields={
        "sql_complexity_description":datapoint['sql_complexity_description'],
        "sql_task_type_description":datapoint['sql_task_type_description'],
        "sql_prompt":datapoint['sql_prompt'],
        "sql_context":datapoint['sql_context'],
        "sql_explnation":datapoint['sql_explanation'],
        "sql":datapoint['sql'],
    }
) for datapoint in t2sql['test']]

In [119]:
len(t2sql_records)

5851

In [120]:
serside_t2sql.add_records(t2sql_records[:15])

d:\argenv\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [20]:
t2sql["test"].select(range(10))

{'id': 1,
 'domain': 'sports',
 'domain_description': 'Extensive data on athlete performance, team management, fan engagement, facility operations, and event planning in sports.',
 'sql_complexity': 'subqueries',
 'sql_complexity_description': 'subqueries, including correlated and nested subqueries',
 'sql_task_type': 'analytics and reporting',
 'sql_task_type_description': 'generating reports, dashboards, and analytical insights',
 'sql_prompt': "Which team has the highest number of wins in the 'basketball_games' table?",
 'sql_context': "CREATE TABLE basketball_teams (team_id INT, name VARCHAR(50)); CREATE TABLE basketball_games (game_id INT, home_team INT, away_team INT, home_team_score INT, away_team_score INT); INSERT INTO basketball_teams (team_id, name) VALUES (1, 'Boston Celtics'), (2, 'Los Angeles Lakers'), (3, 'Chicago Bulls'); INSERT INTO basketball_games (game_id, home_team, away_team, home_team_score, away_team_score) VALUES (1, 1, 2, 85, 80), (2, 2, 3, 95, 90), (3, 3, 1, 

In [135]:
# Sending metadata in bulk under one metadata
t2sql_records = [FeedbackRecord(
    fields={
        "sql_complexity_description": datapoint['sql_complexity_description'],
        "sql_task_type_description": datapoint['sql_task_type_description'],
        "sql_prompt": datapoint['sql_prompt'],
        "sql_context": datapoint['sql_context'],
        "sql_explanation": datapoint['sql_explanation'],
        "sql": datapoint['sql'],
    }, # "domain_description", "sql_complexity", "sql_task_type"
    metadata={
        'domain': datapoint['domain'],
        'domain_description': datapoint['domain_description'],
        'sql_complexity': datapoint['sql_complexity'],
        'sql_task_type': datapoint['sql_task_type'],
    }
) for datapoint in t2sql['test']]

In [136]:
serside_t2sql.add_records(t2sql_records[:15])

In [137]:
updt_vectors = ste.update_records(
    records=serside_t2sql.records,
    fields=None,
    overwrite=True
)

Batches: 100%|██████████| 1/1 [00:09<00:00,  9.92s/it]


In [52]:
from rich import print

# print(updt_vectors[0])

RemoteFeedbackRecord(
    id=UUID('38da753a-df4b-44e2-ab1f-63fd6ce8c621'),
    client=<httpx.Client object at 0x000001C6C530AFD0>,
    fields={
        'sql_complexity_description': 'subqueries, including correlated and nested subqueries',
        'sql_task_type_description': 'generating reports, dashboards, and analytical insights',
        'sql_prompt': "Which team has the highest number of wins in the 'basketball_games' table?",
        'sql_context': "CREATE TABLE basketball_teams (team_id INT, name VARCHAR(50)); CREATE TABLE 
basketball_games (game_id INT, home_team INT, away_team INT, home_team_score INT, away_team_score INT); INSERT INTO
basketball_teams (team_id, name) VALUES (1, 'Boston Celtics'), (2, 'Los Angeles Lakers'), (3, 'Chicago Bulls'); 
INSERT INTO basketball_games (game_id, home_team, away_team, home_team_score, away_team_score) VALUES (1, 1, 2, 85,
80), (2, 2, 3, 95, 90), (3, 3, 1, 75, 85);",
        'sql': 'SELECT name AS team, MAX(home_team_wins + away_team_wins) AS highest_wins FROM (SELECT name, CASE 
WHEN home_team = team_id AND home_team_score > away_team_score THEN 1 ELSE 0 END + CASE WHEN away_team = team_id 
AND away_team_score > home_team_score THEN 1 ELSE 0 END AS home_team_wins, CASE WHEN home_team = team_id AND 
home_team_score < away_team_score THEN 1 ELSE 0 END + CASE WHEN away_team = team_id AND away_team_score < 
home_team_score THEN 1 ELSE 0 END AS away_team_wins FROM basketball_teams JOIN basketball_games ON 
basketball_teams.team_id = basketball_games.home_team OR basketball_teams.team_id = basketball_games.away_team) AS 
subquery GROUP BY name;'
    },
    metadata={
        'domain': 'sports',
        'domain_description': 'Extensive data on athlete performance, team management, fan engagement, facility 
operations, and event planning in sports.',
        'sql_complexity': 'subqueries',
        'sql_task_type': 'analytics and reporting'
    },
    vectors={
        'sql_task_type_description': [
            0.21709346771240234,
            0.1280633807182312,
            0.20399893820285797,
            -0.24313397705554962,
            0.4482356011867523,
            0.1670333445072174,
            -0.07946325838565826,
            0.2602781057357788,
            0.2791512906551361,
            0.22560377418994904,
            -0.06364230066537857,
            -0.08092556893825531,
            0.17910274863243103,
            0.12170560657978058,
            0.10956103354692459,
            0.14922083914279938,
            -0.07232256978750229,
            -0.33666276931762695,
            0.05319720879197121,
            -0.32093608379364014,
            0.050672292709350586,
            -0.10605542361736298,
            0.12682397663593292,
            -0.3467533588409424,
            0.057984936982393265,
            0.4855263829231262,
            -0.1042269915342331,
            -0.20926450192928314,
            -0.45306798815727234,
            -1.1135698556900024,
            -0.05647895112633705,
            -0.22178806364536285,
            0.44888541102409363,
            0.13383843004703522,
            -0.08027923852205276,
            -0.07558277994394302,
            -0.1467130184173584,
            0.39868345856666565,
            0.03206739202141762,
            0.07596950978040695,
            0.1665039211511612,
            -0.053545888513326645,
            -0.1418655961751938,
            -0.03956952318549156,
            -0.1923668384552002,
            -0.5418616533279419,
            -0.0836731418967247,
            0.16731701791286469,
            -0.3765716552734375,
            -0.18909479677677155,
            -0.34156957268714905,
            -0.2943894565105438,
            0.11688321083784103,
            0.1442638337612152,
            0.01953735388815403,
            0.2648825943470001,
            0.3989311456680298,
            0.11964822560548782,
            -0.1027451679110527,
            0.07626371830701828,
            0.5016967058181763,
     

In [138]:
ste.update_dataset(
    dataset=serside_t2sql,
    fields=None,
    update_records=True,
    overwrite=False
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 ste.update_dataset(                                                                          │
│   2 │   dataset=serside_t2sql,                                                                   │
│   3 │   fields=None,                                                                             │
│   4 │   update_records=True,                                                                     │
│                                                                                                  │
│ d:\argenv\Lib\site-packages\argilla\client\feedback\integrations\sentencetransformers.py:222 in  │
│ update_dataset                                                                                   │
│                                                                                                  │
│   219 │   │   if not all([field in available_fields for field in fields]):                       │
│   220 │   │   │   raise ValueError(f"Fields {fields} are not present in the dataset.")           │
│   221 │   │                                                                                      │
│ ❱ 222 │   │   dataset = self._create_vector_settings(dataset=dataset, fields=fields, overwrite   │
│   223 │   │                                                                                      │
│   224 │   │   if update_records:                                                                 │
│   225 │   │   │   records = self.update_records(records=dataset.records, fields=fields, overwr   │
│                                                                                                  │
│ d:\argenv\Lib\site-packages\argilla\client\feedback\integrations\sentencetransformers.py:104 in  │
│ _create_vector_settings                                                                          │
│                                                                                                  │
│   101 │   │   │   │   │   │   │   f"Field {field} has a different embedding dimension ({datase   │
│   102 │   │   │   │   │   │   )                                                                  │
│   103 │   │   │   else:                                                                          │
│ ❱ 104 │   │   │   │   dataset.add_vector_settings(                                               │
│   105 │   │   │   │   │   VectorSettings(                                                        │
│   106 │   │   │   │   │   │   name=field,                                                        │
│   107 │   │   │   │   │   │   dimensions=self.embedding_dim,                                     │
│                                                                                                  │
│ d:\argenv\Lib\site-packages\argilla\client\utils.py:61 in wrapper                                │
│                                                                                                  │
│    58 │   │   │   │   │   f"User with role={user.role} is not allowed to call `{func.__name__}   │
│    59 │   │   │   │   │   f" Only users with role={roles} are allowed to call this function."    │
│    60 │   │   │   │   )                                                                          │
│ ❱  61 │   │   │   return func(*args, **kwargs)                                                   │
│    62 │   │                                                                                      │
│    63 │   │   return wrapper                                                                     │
│    64                                                                                            │
│                                                            

In [148]:
serside_t2sql.push_to_huggingface(repo_id="Kamaljp/text2sql_argilla")

d:\argenv\Lib\site-packages\argilla\client\feedback\dataset\remote\dataset.py:1018: UserWarning: The dataset is first pulled locally and pushed to Hugging Face after because `push_to_huggingface` is not supported for a `RemoteFeedbackDataset`. `RemoteFeedbackDataset.pull().push_to_huggingface(...)` is applied.
  warnings.warn(


Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.20s/it]


In [149]:
text2sql_fromhf = FeedbackDataset.from_huggingface("Kamaljp/text2sql_argilla")
text2sql_fromhf

Generating train split: 15 examples [00:00, 85.65 examples/s]
Parsing records: 100%|██████████| 15/15 [00:01<00:00,  8.23it/s]


FeedbackDataset(
   fields=[TextField(name='sql_complexity_description', title='Sql Complexity', required=True, type=<FieldTypes.text: 'text'>, use_markdown=True), TextField(name='sql_task_type_description', title='Task Description', required=True, type=<FieldTypes.text: 'text'>, use_markdown=True), TextField(name='sql_prompt', title='prompt', required=True, type=<FieldTypes.text: 'text'>, use_markdown=True), TextField(name='sql_context', title='context', required=True, type=<FieldTypes.text: 'text'>, use_markdown=True), TextField(name='sql', title='SQL Query', required=True, type=<FieldTypes.text: 'text'>, use_markdown=True)]
   questions=[TextQuestion(name='sqltext', title='Checking output of the sql query, sql explanation', description='Review the SQL query field and provide feedback', required=True, type=<QuestionTypes.text: 'text'>, use_markdown=True)]
   guidelines=None)
   metadata_properties=[TermsMetadataProperty(name='domain', title='domain of prompt', visible_for_annotators=

In [139]:
# preparing the vector setting for the datasets
from argilla import VectorSettings
vector_set_domain = VectorSettings(
    name="domain",
    title="domain vectorised",
    dimensions=384
)

vector_set_desc = VectorSettings(
    name="domain_description",
    title="domain description vectorised",
    dimensions=384
)

In [140]:
# including the vector settings

from argilla import (
    FeedbackDataset,
    FeedbackRecord,
    TextField,
)
# sql_complexity_description, sql_task_type_description, sql_prompt, sql_context
t2sql_fbds = FeedbackDataset(
    fields=[
        TextField(name='sql_complexity_description', title='Sql Complexity', required=True, use_markdown=True),
        TextField(name='sql_task_type_description', title='Task Description', required=True, use_markdown=True),
        TextField(name='sql_prompt', title='prompt', required=True, use_markdown=True),
        TextField(name='sql_context', title='context', required=True, use_markdown=True),
        TextField(name='sql', title='SQL Query', required=True, use_markdown=True),
    ],
    questions=[quest],
    # metadata_properties=[meta],  # If there is no metadata properties, not reflected in UI
    metadata_properties=[meta_domain, meta_domain_desc, meta_sql_complexity, meta_sql_task],
    vectors_settings=[vector_set_domain, vector_set_desc],
    allow_extra_metadata=True
)

In [143]:
t2sql_fbds.push_to_argilla(name="t2sql_ds", workspace=hfgilla)

[04/06/24 16:24:06] INFO     INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully  ]8;id=780311;file://d:\argenv\Lib\site-packages\argilla\client\feedback\dataset\local\mixins.py\mixins.py]8;;\:]8;id=55266;file://d:\argenv\Lib\site-packages\argilla\client\feedback\dataset\local\mixins.py#271\271]8;;\
                             pushed to Argilla                                                                     

                    INFO     INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset( ]8;id=275442;file://d:\argenv\Lib\site-packages\argilla\client\feedback\dataset\local\mixins.py\mixins.py]8;;\:]8;id=10850;file://d:\argenv\Lib\site-packages\argilla\client\feedback\dataset\local\mixins.py#272\272]8;;\
                                id=0c3c295b-effd-4178-916b-5eb1938346c2                                            
                                name=t2sql_ds                                                                      
                                workspace=Workspace(id=eb6c560d-627d-42e1-bdbf-ace020199880,                       
                             name=hfgilla, inserted_at=2024-04-06 04:22:46.479068,                                 
                             updated_at=2024-04-06 04:22:46.479068)                                                
                                url=https://kamaljp-argillatest.hf.space/dataset/0c3c295b-effd-4178-9              
                             16b-5eb1938346c2/annotation-mode                                                      
                                fields=[RemoteTextField(id=UUID('008b977a-dd70-44da-9bcf-8282b55c3fc3              
                             '), client=None, name='sql_complexity_description', title='Sql                        
                             Complexity', required=True, type='text', use_markdown=True),                          
                             RemoteTextField(id=UUID('9f347273-4ddc-45a9-8087-f76a72a37895'),                      
                             client=None, name='sql_task_type_description', title='Task Description',              
                             required=True, type='text', use_markdown=True),                                       
                             RemoteTextField(id=UUID('949906d9-12ae-4329-a87e-088c57357a38'),                      
                             client=None, name='sql_prompt', title='prompt', required=True,                        
                             type='text', use_markdown=True),                                                      
                             RemoteTextField(id=UUID('2fc48e9e-1120-4a27-9a9e-d1810e6f69fb'),                      
                             client=None, name='sql_context', title='context', required=True,                      
                             type='text', use_markdown=True),                                                      
                             RemoteTextField(id=UUID('13be500d-0697-48d6-8a8d-848500beba76'),                      
                             client=None, name='sql', title='SQL Query', required=True, type='text',               
                             use_markdown=True)]                                                                   
                                questions=[RemoteTextQuestion(id=UUID('f3c3eb57-ad13-408f-84cf-cff531              
                             c489ed'), client=None, name='sqltext', title='Checking output of the sql              
                             query, sql explanation', description='Review the SQL query field and                  
                             provide feedback', required=True, type='text', use_markdown=True)]                    
                                guidelines=None                                                                    
                                metadata_properties=[RemoteTermsMetadataProperty(id=UUID('686f2fc0-1a              
                             c5-4171-9201-498c07c6341e'), client=<httpx.Client object at                           
                             0x000001C6A5AB7690>, name='domain', title='domain of prompt',                         
                             visible_for_annotators=True, type='terms', values=None),                              
                             RemoteTermsMetadataProperty(id=UUID('f214564d-

RemoteFeedbackDataset(
   id=0c3c295b-effd-4178-916b-5eb1938346c2
   name=t2sql_ds
   workspace=Workspace(id=eb6c560d-627d-42e1-bdbf-ace020199880, name=hfgilla, inserted_at=2024-04-06 04:22:46.479068, updated_at=2024-04-06 04:22:46.479068)
   url=https://kamaljp-argillatest.hf.space/dataset/0c3c295b-effd-4178-916b-5eb1938346c2/annotation-mode
   fields=[RemoteTextField(id=UUID('008b977a-dd70-44da-9bcf-8282b55c3fc3'), client=None, name='sql_complexity_description', title='Sql Complexity', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('9f347273-4ddc-45a9-8087-f76a72a37895'), client=None, name='sql_task_type_description', title='Task Description', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('949906d9-12ae-4329-a87e-088c57357a38'), client=None, name='sql_prompt', title='prompt', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('2fc48e9e-1120-4a27-9a9e-d1810e6f69fb'), client=None, name='sql_context', title='context

In [144]:
# run delete only when required
serside_t2sql = FeedbackDataset.from_argilla(name="t2sql_ds", workspace=hfgilla)

In [142]:
# run delete only when required
serside_t2sql.delete()

In [64]:
from sentence_transformers import SentenceTransformer
tformer = SentenceTransformer(model_name_or_path="all-MiniLM-L6-v2")

In [65]:
tformer

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [145]:
example_vector = tformer.encode("This is the example vectorisation")
print(type(example_vector))
len(example_vector)

<class 'numpy.ndarray'>

384

In [146]:
# Sending metadata in bulk under one metadata
t2sql_records = [FeedbackRecord(
    fields={
        "sql_complexity_description": datapoint['sql_complexity_description'],
        "sql_task_type_description": datapoint['sql_task_type_description'],
        "sql_prompt": datapoint['sql_prompt'],
        "sql_context": datapoint['sql_context'],
        "sql": datapoint['sql'],
    }, # "domain_description", "sql_complexity", "sql_task_type"
    metadata={
        'domain': datapoint['domain'],
        'domain_description': datapoint['domain_description'],
        'sql_complexity': datapoint['sql_complexity'],
        'sql_task_type': datapoint['sql_task_type'],
    },
    vectors={
            "domain": list(tformer.encode(datapoint['domain'])),
            "domain_description": list(tformer.encode(datapoint['domain_description'])),
        },
) for datapoint in t2sql['test'].select(range(15))]

In [147]:
serside_t2sql.add_records(t2sql_records)